In [36]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import math
import numpy as np
import torch.nn.functional as F

#import matplotlib as plt

In [14]:
class Embeddings(nn.Module):
    def __init__(self, dim, vocab_size):
        super(Embeddings,self).__init__() # == super().__init__() 
        self.lut = nn.Embedding(vocab_size,dim)
        self.d_model = dim
    
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [15]:
def subsequent_mask(size):
    attention_size = (1,size,size)
    subseq_mask = np.triu(np.ones(attention_size),k=1)
    return torch.from_numpy(1-subseq_mask)

In [16]:
trid = subsequent_mask(6)
trid,trid.shape

(tensor([[[1., 0., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0.],
          [1., 1., 1., 0., 0., 0.],
          [1., 1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1., 1.]]], dtype=torch.float64),
 torch.Size([1, 6, 6]))

In [17]:
trid[0].shape

torch.Size([6, 6])

In [18]:
embedding = nn.Embedding(10,3)
type(embedding)


torch.nn.modules.sparse.Embedding

In [19]:
embedding.weight

Parameter containing:
tensor([[ 0.1325,  1.1871,  0.6789],
        [ 1.0420,  0.9563,  0.3116],
        [-0.6798, -1.5014,  0.9855],
        [-1.0153, -0.9032, -1.0488],
        [-0.1609, -0.3015, -0.2394],
        [-1.6616,  0.2003, -0.2897],
        [-1.7343,  0.7503, -0.8672],
        [-2.0518,  0.5039, -0.2504],
        [-0.4660,  1.0216,  1.1099],
        [-0.5685,  0.0497, -1.0589]], requires_grad=True)

In [20]:
inputs = torch.LongTensor([[1,2,3,4],[0,6,7,8]])
inputs.shape

torch.Size([2, 4])

In [21]:
emb = embedding(inputs)

In [22]:
emb

tensor([[[ 1.0420,  0.9563,  0.3116],
         [-0.6798, -1.5014,  0.9855],
         [-1.0153, -0.9032, -1.0488],
         [-0.1609, -0.3015, -0.2394]],

        [[ 0.1325,  1.1871,  0.6789],
         [-1.7343,  0.7503, -0.8672],
         [-2.0518,  0.5039, -0.2504],
         [-0.4660,  1.0216,  1.1099]]], grad_fn=<EmbeddingBackward0>)

In [23]:
emb.shape

torch.Size([2, 4, 3])

In [24]:
x = Variable(torch.LongTensor([[1,2,3,4],[0,6,7,8]]))

In [25]:
x

tensor([[1, 2, 3, 4],
        [0, 6, 7, 8]])

In [26]:
type(x),type(inputs)

(torch.Tensor, torch.Tensor)

In [27]:
d_model =512
vocab_size = 10000
embs = Embeddings(d_model,vocab_size= vocab_size)

In [28]:
inputs_emb = embs(x)

In [37]:
inputs_emb.shape[-1],inputs_emb.size(),inputs_emb.size()[-1]

(512, torch.Size([2, 4, 512]), 512)

In [41]:
def attention(query, key, value, mask = None, dropout = None):
    d_k = query.size()[-1]
    scores = torch.matmul(query, key.transpose(-2,-1))/math.sqrt(d_k) #key last dim exchange with last 2nd dim
    print(scores.size())
    if mask:
        scores = scores.masked_fill(mask==0, -1e9)
    attn = F.softmax(scores, dim = -1)
    print(attn.size())
    if dropout:
        attn = dropout(attn)
    return torch.matmul(attn, value), attn

In [44]:
query = key = value = inputs_emb

In [45]:
value.size()

torch.Size([2, 4, 512])

In [46]:
values, attn = attention(query,key,value)

torch.Size([2, 4, 4])
torch.Size([2, 4, 4])


In [49]:
values.shape

torch.Size([2, 4, 512])

In [54]:
attn.shape

torch.Size([2, 4, 4])

In [55]:
import copy
def clones(module, n = 1):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(n)])


In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, head, embedding_dim, dropout = 0.1):
        super().__init__()
        assert embedding_dim % head == 0
        self.d_k = embedding_dim // head
        self.head = head
        
        